# Flash Evaluation on CamFow Dataset:

This notebook is dedicated to evaluating Flash on the 5G CamFlow datasets, which are graph-level in nature. We employ Flash in graph-level detection mode to analyze this dataset effectively. Upon completion of the notebook execution, the results will be presented.

## Dataset Access: 
- This dataset will be publically available upon publishing

## Data Parsing and Execution:
- Utilize the parser included in this notebook to process the downloaded files.
- To obtain the evaluation results, execute all cells within this notebook.

## Model Training and Execution Flexibility:
- By default, the notebook operates using pre-trained model weights.
- Additionally, this notebook offers the flexibility to set parameters for training Graph Neural Networks (GNNs) and word2vec models from scratch.
- You can then utilize these freshly trained models to conduct the evaluation. 

Follow these guidelines for a thorough and efficient analysis of the Unicorn datasets using Flash.


In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
from torch_geometric.loader import NeighborLoader
import multiprocessing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline

In [2]:
from pprint import pprint
import gzip
from sklearn.manifold import TSNE
import json
import copy
import os
from tqdm import tqdm, trange

In [3]:
Train_Gnn = False
Train_Word2vec = False
Parse_data = False
Disable_tqdm = True

In [4]:
from tqdm import tqdm 
def show(str):
	print (str + ' ' + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time())))
def prepare_graph(df):
    def process_node(node, action, node_dict, label_dict, dummies, node_type):
        node_dict.setdefault(node, []).append(action)
        label_dict[node] = dummies.get(getattr(row, node_type), -1)  

    nodes = {}
    labels = {}
    edges = []
    # dummies = {
    #     "7998762093665332071": 0,
    #     "14709879154498484854": 1,
    #     "10991425273196493354": 2,
    #     "14871526952859113360": 3,
    #     "8771628573506871447": 4,
    #     "7877121489144997480": 5,
    #     "17841021884467483934": 6,
    #     "7895447931126725167": 7,
    #     "15125250455093594050": 8,
    #     "8664433583651064836": 9,
    #     "14377490526132269506": 10,
    #     "15554536683409451879": 11,
    #     "8204541918505434145": 12,
    #     "14356114695140920775": 13
    # }
    
    dummies = {'address': 0,
        'argv': 1,
        'block': 2,
        'file': 3,
        'iattr': 4,
        'link': 5,
        'path': 6,
        'pipe': 7,
        'process_memory': 8,
        'socket': 9,
        'task': 10,
        'xattr': 11}
    
    # dummies = {'block': 0,
    #     'file': 1,
    #     'iattr': 2,
    #     'link': 3,
    #     'path': 4,
    #     'pipe': 5,
    #     'process_memory': 6,
    #     'socket': 7,
    #     'task': 8}

    for row in df.itertuples():
        process_node(row.actorID, row.action, nodes, labels, dummies, 'actor_type')
        process_node(row.objectID, row.action, nodes, labels, dummies, 'object')

        edges.append((row.actorID, row.objectID))

    features = [nodes[node] for node in tqdm(nodes, disable=Disable_tqdm)]
    feat_labels = [labels[node] for node in tqdm(nodes, disable=Disable_tqdm)]
    edge_index = [[], []]
    node_index_map = {node: i for i, node in enumerate(nodes.keys())}
    for src, dst in tqdm(edges, disable=Disable_tqdm):
        src_index = node_index_map[src]
        dst_index = node_index_map[dst]
        edge_index[0].append(src_index)
        edge_index[1].append(dst_index)

    return features, feat_labels, edge_index, list(nodes.keys())


In [5]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv, GATConv
import torch.nn.functional as F
import torch.nn as nn

class GCN(torch.nn.Module):
    def __init__(self,in_channel,out_channel):
        super().__init__()
        self.conv1 = SAGEConv(in_channel, 32, normalize=True)
        self.conv2 = SAGEConv(32, 20, normalize=True)
        self.linear = nn.Linear(in_features=20,out_features=out_channel)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        x = self.linear(x)
        return F.softmax(x, dim=1)

In [6]:
from gensim.models.callbacks import CallbackAny2Vec
import gensim
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm


class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        model.save('trained_weights/5gcamflow/5gcamflow.model')
        self.epoch += 1

In [7]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [8]:
logger = EpochLogger()
saver = EpochSaver()
date = '30-04-2024'

In [9]:
if Train_Word2vec:
    comb_data = []
    for i in tqdm(range(100)):
        f = open(f"5gcamflow/{date}/{i}.txt")
        data = f.read().split('\n')
        data = [line.split('\t') for line in data]
        comb_data = comb_data + data

    df = pd.DataFrame (comb_data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()
    phrases,labels,edges,mapp = prepare_graph(df)
    


In [10]:
if Train_Word2vec:
        word2vec = Word2Vec(sentences=phrases, vector_size=30, window=5, min_count=1, workers=32,epochs=300,callbacks=[saver,logger])

In [11]:
from sklearn.utils import class_weight
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

model = GCN(30,14).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [12]:
from collections import Counter
import math

class PositionalEncoder:

    def __init__(self, d_model, max_len=100000, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        position = torch.arange(max_len, device=device).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2, device=device) * (-math.log(10000.0) / d_model))
        self.pe = torch.zeros(max_len, d_model, device=device)
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)

    def embed(self, x):
        return x + self.pe[:x.size(0)]

def infer(document):
    word_embeddings = [w2vmodel.wv[word] for word in document if word in  w2vmodel.wv]
    
    if not word_embeddings:
        return np.zeros(30)

    output_embedding = torch.tensor(word_embeddings, dtype=torch.float, device=device)
    if len(document) < 100000:
        output_embedding = encoder.embed(output_embedding)

    output_embedding = output_embedding.detach().cpu().numpy()
    return np.mean(output_embedding, axis=0)

encoder = PositionalEncoder(30)
w2vmodel = Word2Vec.load("trained_weights/5gcamflow/5gcamflow.model")

In [13]:
train_stop = 250
val_start = train_stop
val_size = 50
val_stop = val_start + val_size
ben_start = val_stop + 1
ben_size = 55
ben_stop = ben_start + ben_size
mal_start = 401
mal_size = 3
mal_stop = mal_start + mal_size


In [14]:
from torch_geometric import utils

################################## Training Main Model #####################################
if Train_Gnn:
    for i in trange(train_stop):
        f = open(f"5gcamflow/{date}/{i}.txt")
        data = f.read().split('\n')

        data = [line.split('\t') for line in data]
        df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
        df.sort_values(by='timestamp', ascending=True,inplace=True)
        df = df.dropna()
        phrases,labels,edges,mapp = prepare_graph(df)

        criterion = CrossEntropyLoss()

        nodes = [infer(x) for x in tqdm(phrases, desc='Inferring Phrases', disable=Disable_tqdm)]
        nodes = np.array(nodes)  

        graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
        graph.n_id = torch.arange(graph.num_nodes)
        mask = torch.tensor([True]*graph.num_nodes, dtype=torch.bool)

        for m_n in range(20):
            loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
            total_loss = 0
            for subg in loader:
                subg.to(device)
                model.train()
                optimizer.zero_grad() 
                out = model(subg.x, subg.edge_index) 
                loss = criterion(out, subg.y) 
                loss.backward() 
                optimizer.step()      
                total_loss += loss.item() * subg.batch_size

            loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
            for subg in loader:
                subg.to(device)
                model.eval()
                out = model(subg.x, subg.edge_index)
                sorted, indices = out.sort(dim=1,descending=True)
                conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
                conf = (conf - conf.min()) / conf.max()
                pred = indices[:,0]
                cond = (pred == subg.y)
                mask[subg.n_id[cond]] = False

            print(f'Model# {m_n}. {mask.sum().item()} nodes still misclassified out of {mask.size()} ')
            torch.save(model.state_dict(), f'trained_weights/5gcamflow/5gcamflow{m_n}.pth')

### Validation

In [15]:
from torch.utils.data import Dataset, DataLoader
class PhraseDataset(Dataset):
    def __init__(self, phrases):
        self.phrases = phrases

    def __len__(self):
        return len(self.phrases)

    def __getitem__(self, idx):
        return self.phrases[idx]

def batch_infer(phrases):
    return [infer(phrase) for phrase in phrases] 

In [16]:
from tqdm import tqdm

scores = []
for i in trange(val_start,val_stop):
    print(f"Graph #: {i}")
    f = open(f"5gcamflow/{date}/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()

    # print("Preparing Graph")
    phrases,labels,edges,mapp = prepare_graph(df)
    # print("Done")
    # print("Inferring words")
    # dataset = PhraseDataset(phrases)
    # loader = DataLoader(dataset, batch_size=10, num_workers=4)
    # nodes = []
    # for batch in tqdm(loader):
    #     batch_embeddings = batch_infer(batch)
    #     nodes.extend(batch_embeddings)
    nodes = [infer(x) for x in tqdm(phrases, disable=Disable_tqdm)]
    nodes = np.array(nodes)  
    # print("Done!")

    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes).to(device)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool).to(device)
    # print("Evaluating...")
    for m_n in tqdm(range(20), disable=Disable_tqdm):
        model.load_state_dict(torch.load(f'trained_weights/5gcamflow/5gcamflow{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y).to(device)
        falses = torch.tensor([False] * len(flag[graph.n_id[cond]]), dtype=torch.bool).to(device)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], falses)
    # print("Done!")
            
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100)
    scores.append(flag.sum().item())

  0%|          | 0/50 [00:00<?, ?it/s]

Graph #: 250


  2%|▏         | 1/50 [00:03<02:40,  3.27s/it]

### Testing

In [54]:
from math import ceil
thresh = 110
scores.sort()
print(scores)
print(len(scores))
avg = sum(scores) / len(scores)
print(avg)

thresh = scores[ceil(len(scores)*0.9)]
print(thresh)

[444, 457, 486, 499, 504, 515, 516, 518, 521, 522, 523, 541, 544, 547, 556, 561, 562, 569, 574, 607, 612, 618, 628, 628, 640, 645, 670, 682, 686, 687, 693, 697, 698, 719, 726, 733, 734, 736, 760, 761, 769, 776, 778, 828, 834, 836, 840, 882, 899, 945]
50
654.12
836


In [55]:
# thresh = 400
correct_benign = 0

for i in trange(ben_start ,ben_stop):
    print(f"Graph #: {i}")
    f = open(f"5gcamflow/{date}/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()

    phrases,labels,edges,mapp = prepare_graph(df)

    nodes = [infer(x) for x in tqdm(phrases, disable=Disable_tqdm)]
    nodes = np.array(nodes)  

    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes).to(device)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool).to(device)

    for m_n in range(20):
        model.load_state_dict(torch.load(f'trained_weights/5gcamflow/5gcamflow{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y).to(device)
        falses = torch.tensor([False]*len(flag[graph.n_id[cond]]), dtype=torch.bool).to(device)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], falses)

    if flag.sum().item() <= thresh:
        correct_benign = correct_benign + 1
            
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100, correct_benign, i)
print(correct_benign)

  0%|          | 0/55 [00:00<?, ?it/s]

Graph #: 301


  2%|▏         | 1/55 [00:14<12:48, 14.23s/it]

688 0.5557395455536798 1 301
Graph #: 302


  4%|▎         | 2/55 [00:28<12:36, 14.27s/it]

...

582 0.4686707306270686 49 353
Graph #: 354


 98%|█████████▊| 54/55 [12:52<00:14, 14.13s/it]

592 0.49187410681644456 50 354
Graph #: 355


100%|██████████| 55/55 [13:06<00:00, 14.30s/it]

766 0.6231492629592268 51 355
51


In [56]:
correct_attack = 0

for i in trange(mal_start,mal_stop):
    print(f"Graph #: {i}")
    f = open(f"5gcamflow/{date}/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()
    
    phrases,labels,edges,mapp = prepare_graph(df)

    nodes = [infer(x) for x in phrases]
    nodes = np.array(nodes)  
    
    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes).to(device)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool).to(device)

    for m_n in range(20):
        model.load_state_dict(torch.load(f'trained_weights/5gcamflow/5gcamflow{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y).to(device)
        falses = torch.tensor([False]*len(flag[graph.n_id[cond]]), dtype=torch.bool).to(device)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], falses)

    if  flag.sum().item() > thresh:
        correct_attack = correct_attack + 1
   
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100, correct_attack, i)

  0%|          | 0/3 [00:00<?, ?it/s]

Graph #: 401


 33%|███▎      | 1/3 [00:13<00:27, 13.54s/it]

1155 0.9822430860291868 1 401
Graph #: 402


 67%|██████▋   | 2/3 [00:27<00:13, 13.85s/it]

1311 1.0594966784657904 2 402
Graph #: 403


100%|██████████| 3/3 [00:38<00:00, 12.91s/it]

1105 1.1380137797505638 3 403


In [57]:
TP = correct_attack
FP = ben_size - correct_benign
TN = correct_benign
FN = mal_size - correct_attack

FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
TPR = TP / (TP + FN) if (TP + FN) > 0 else 0

print(f"Number of True Positives (TP): {TP}")
print(f"Number of False Positives (FP): {FP}")
print(f"Number of False Negatives (FN): {FN}")
print(f"Number of True Negatives (TN): {TN}\n")

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TPR  
accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

fscore = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f"Fscore: {fscore}\n")

Number of True Positives (TP): 3
Number of False Positives (FP): 4
Number of False Negatives (FN): 0
Number of True Negatives (TN): 51

Precision: 0.42857142857142855
Recall: 1.0
Fscore: 0.6

